In [5]:
!pip install streamlit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
%%writefile app.py
import streamlit as st
import numpy as np
import cv2
import random
from PIL import Image
import io
#@st.cache(allow_output_mutation=True) #Guarda datos mientras se avanza
@st.cache_resource
def crear_art_PB(img,num_circulos):
  radios = np.random.randint(1, 2, num_circulos)
  posiciones = np.random.randint(0, img.shape[0], (num_circulos, 2))
  artefactos = np.zeros_like(img)
  for i in range(num_circulos):
    cv2.circle(artefactos, tuple(posiciones[i]), radios[i], (255, 255, 255), -1)
  img_con_artefactos = cv2.add(img, artefactos)
  return img_con_artefactos

def add_noise(image, sigma):
    noise = np.random.normal(scale=sigma, size=image.shape)
    noisy_image = image + noise
    noisy_image = cv2.normalize(noisy_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return noisy_image
def aplic(imag, aa):
   if aa=='Ninguno':
      imag=imag
   elif aa=='Pixel Blanco':
      imag=crear_art_PB(imag, 15)
   elif aa=='Artefacto':
      imag = cv2.GaussianBlur(imag, (15,15), 0)
      imag=add_noise(imag, 25)
   elif aa== 'Combinado':
      imag = cv2.GaussianBlur(imag, (15,15), 0)
      imag=add_noise(imag, 25)
      imag=crear_art_PB(imag, 15)
   return imag
    
   
def main():
   st.title("SAIM by MEDImages")
   imag1 =st.sidebar.file_uploader("Carga de imagen", type=["jpg"])

   sa=st.sidebar.radio("Tipo de artefactos", ["Ninguno", "Pixel Blanco", "Artefacto", "Combinado"])


   if imag1 is not None:
      imag2= cv2.imdecode(np.fromstring(imag1.read(),np.uint8),1)
      c1,c2=st.columns(2)
      c1.image(imag2,caption="Original",use_column_width=True)
      imr=aplic(imag2, sa)
      c2.image(imr,caption="Imagen con artefactos aplicados",use_column_width=True)
      with io.BytesIO() as output:
       img_pil = Image.fromarray(imr)
       img_pil.save(output, format='JPEG')
       img_bytes = output.getvalue()
      st.download_button(label="Descarga", data=img_bytes, file_name="image_art.jpg")

if __name__=='__main__':
   main()


Overwriting app.py


In [7]:
!ls

app.py	sample_data


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



npx: installed 22 in 3.542s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.206.199:8501

your url is: https://ten-beds-look.loca.lt
